In [1]:
import pandas as pd
import censusdata
import openpyxl
import re
import os
import numpy

In [2]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [3]:
censusdata.search('acs1', 2012, 'concept', 'race')

[('B01001F_001E', 'SEX BY AGE (SOME OTHER RACE ALONE)', 'Estimate!!Total'),
 ('B01001F_002E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male'),
 ('B01001F_003E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!Under 5 years'),
 ('B01001F_004E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!5 to 9 years'),
 ('B01001F_005E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!10 to 14 years'),
 ('B01001F_006E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!15 to 17 years'),
 ('B01001F_007E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!18 and 19 years'),
 ('B01001F_008E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!20 to 24 years'),
 ('B01001F_009E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!25 to 29 years'),
 ('B01001F_010E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!30 to 34 years'),
 ('B01001F_011E',
  'SEX

In [4]:
censusdata.search('acs1', 2012, 'label', 'age')[0]

('B01002A_001E',
 'MEDIAN AGE BY SEX (WHITE ALONE)',
 'Estimate!!Median age!!Total')

In [5]:
censusdata.printtable(censusdata.censustable('acs1', 2012, 'B01001'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B01001_001E  | SEX BY AGE                     | !! Estimate Total                                        | int  
B01001_002E  | SEX BY AGE                     | !! !! Estimate Total Male                                | int  
B01001_003E  | SEX BY AGE                     | !! !! !! Estimate Total Male Under 5 years               | int  
B01001_004E  | SEX BY AGE                     | !! !! !! Estimate Total Male 5 to 9 years                | int  
B01001_005E  | SEX BY AGE                     | !! !! !! Estimate Total Male 10 to 14 years              | int  
B01001_006E  | SEX BY AGE                     | !! !! !! Estimate Total Male 15 to 17 years              | int  
B01001_007E  | SEX BY AGE                     | !! !! !! Estimate Total Male 18 and 19 years 

In [6]:
censusdata.printtable(censusdata.censustable('acs1', 2012, 'B01002A'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B01002A_001E | MEDIAN AGE BY SEX (WHITE ALONE | !! !! Estimate Median age Total                          | float
B01002A_002E | MEDIAN AGE BY SEX (WHITE ALONE | !! !! Estimate Median age Male                           | float
B01002A_003E | MEDIAN AGE BY SEX (WHITE ALONE | !! !! Estimate Median age Female                         | float
-------------------------------------------------------------------------------------------------------------------


In [7]:
censusdata.geographies(censusdata.censusgeo([('state', '*')]), 'acs1', 2012)

{'Alabama': censusgeo((('state', '01'),)),
 'Alaska': censusgeo((('state', '02'),)),
 'Arizona': censusgeo((('state', '04'),)),
 'Arkansas': censusgeo((('state', '05'),)),
 'California': censusgeo((('state', '06'),)),
 'Colorado': censusgeo((('state', '08'),)),
 'Delaware': censusgeo((('state', '10'),)),
 'District of Columbia': censusgeo((('state', '11'),)),
 'Connecticut': censusgeo((('state', '09'),)),
 'Florida': censusgeo((('state', '12'),)),
 'Georgia': censusgeo((('state', '13'),)),
 'Idaho': censusgeo((('state', '16'),)),
 'Hawaii': censusgeo((('state', '15'),)),
 'Illinois': censusgeo((('state', '17'),)),
 'Indiana': censusgeo((('state', '18'),)),
 'Iowa': censusgeo((('state', '19'),)),
 'Kansas': censusgeo((('state', '20'),)),
 'Kentucky': censusgeo((('state', '21'),)),
 'Louisiana': censusgeo((('state', '22'),)),
 'Maine': censusgeo((('state', '23'),)),
 'Maryland': censusgeo((('state', '24'),)),
 'Massachusetts': censusgeo((('state', '25'),)),
 'Michigan': censusgeo((('stat

In [8]:
censusdata.geographies(censusdata.censusgeo([('state', '36'), ('state%20legislative%20district%20(upper%20chamber)', '*')]), 'acs5', 2012)


{'State Senate District 40 (2012), New York': censusgeo((('state', '36'), ('state legislative district (upper chamber)', '040'))),
 'State Senate District 8 (2012), New York': censusgeo((('state', '36'), ('state legislative district (upper chamber)', '008'))),
 'State Senate District 39 (2012), New York': censusgeo((('state', '36'), ('state legislative district (upper chamber)', '039'))),
 'State Senate District 9 (2012), New York': censusgeo((('state', '36'), ('state legislative district (upper chamber)', '009'))),
 'State Senate District 10 (2012), New York': censusgeo((('state', '36'), ('state legislative district (upper chamber)', '010'))),
 'State Senate District 57 (2012), New York': censusgeo((('state', '36'), ('state legislative district (upper chamber)', '057'))),
 'State Senate District 58 (2012), New York': censusgeo((('state', '36'), ('state legislative district (upper chamber)', '058'))),
 'State Senate District 59 (2012), New York': censusgeo((('state', '36'), ('state leg

In [9]:
data = censusdata.download('acs5', 2012, censusdata.censusgeo([('state', '36'), ('state%20legislative%20district%20(upper%20chamber)', '*')]), ['B01001_001E', 'B01001_002E', 'B01001_026E'])

In [10]:
data

,B01001_001E,B01001_002E,B01001_026E
"State Senate District 40 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):040",304418,150390,154028
"State Senate District 8 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):008",312213,150938,161275
"State Senate District 39 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):039",293597,145730,147867
"State Senate District 9 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):009",312806,151273,161533
"State Senate District 10 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):010",323789,151850,171939
...,...,...,...
"State Senate District 34 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):034",321252,157436,163816
"State Senate District 35 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):035",301542,145250,156292
"State Senate District 36 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):036",325191,146459,178732
"State Senate District 37 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):037",314509,151213,163296


In [11]:
values_to_grab = {'B01001001': "Total Pop",
                  'B01001002': "Male",
                  'B01001026': "Female",
                  'B02001002':"White alone",
                  'B02001003':"Black or African American alone",
                  'B02001004':"American Indian and Alaska Native alone",
                  'B02001005':"Asian alone",
                  'B02001006':"Native Hawaiian and Other Pacific Islander alone",
                  'B02001007':"Some other race alone",
                  'B02001008':"Two or more races:",
                  'B02001009':"Two races including Some other race",
                  'B02001010':"Two races excluding Some other race, and three or more races",
                  'B05001002':"U.S. citizen, born in the United States",
                  'B05001003':"U.S. citizen, born in Puerto Rico or U.S. Island Areas",
                  # Universe is limited to families with children
                  # 'B05010002':"Under poverty level",
                  # 'B05010010':"Between 1x and 1.99x poverty level",
                  # 'B05010018':"At least 2x poverty level",
                  "B06007002"	:"Speak only English",
                  "B06007003"	:"Speak Spanish",
                  "B06007006"	:"Speak other languages",
                  "B06008002"	:"Never married",
                  "B06008003"	:"Now married, except separated",
                  "B06008004"	:"Divorced",
                  "B06008005"	:"Separated",
                  "B06008006"	:"Widowed",
                  "B06009002"	:"Less than high school graduate",
                  "B06009003"	:"High school graduate (includes equivalency)",
                  "B06009004"	:"Some college or associate's degree",
                  "B06009005"	:"Bachelor's degree",
                  "B06009006"	:"Graduate or professional degree",
                  "B06010002"	:"No income",
                  "B06010004"	:"$1 to $9,999 or loss",
                  "B06010005"	:"$10,000 to $14,999",
                  "B06010006"	:"$15,000 to $24,999",
                  "B06010007"	:"$25,000 to $34,999",
                  "B06010008"	:"$35,000 to $49,999",
                  "B06010009"	:"$50,000 to $64,999",
                  "B06010010"	:"$65,000 to $74,999",
                  "B06010011"	:"$75,000 or more",
                  "B16004002"	:"5 to 17 years",
                  "B16004024"	:"18 to 64 years",
                  "B16004046"	:"65 years and over"








}

In [12]:
ny_senate_demo_data_2012 = censusdata.download('acs5', 2012, censusdata.censusgeo([('state', '36'), ('state%20legislative%20district%20(upper%20chamber)', '*')]), [each[:6] + '_' + each[6:] + 'E' for each in values_to_grab.keys()])

In [13]:
ny_senate_demo_data_2012 = ny_senate_demo_data_2012.rename(columns=dict(zip([each[:6] + '_' + each[6:] + 'E' for each in values_to_grab.keys()], values_to_grab.values())))

ny_senate_demo_data_2012

,Total Pop,Male,Female,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races:,...,"$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $64,999","$65,000 to $74,999","$75,000 or more",5 to 17 years,18 to 64 years,65 years and over
"State Senate District 40 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):040",304418,150390,154028,253024,15252,622,10588,27,17849,7056,...,14996,25047,21186,23325,20229,10449,62704,57746,188432,42036
"State Senate District 8 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):008",312213,150938,161275,227510,50463,934,9875,9,16133,7289,...,17663,27642,24005,29723,23567,11550,48856,55467,195262,43341
"State Senate District 39 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):039",293597,145730,147867,229963,29964,689,7720,20,18671,6570,...,18094,27908,22690,24429,21008,10466,35704,59077,180984,31151
"State Senate District 9 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):009",312806,151273,161533,232591,37889,644,18038,9,17437,6198,...,16376,26816,21828,27145,25529,11300,57567,55381,194032,47158
"State Senate District 10 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):010",323789,151850,171939,48425,169970,1496,40305,141,50220,13232,...,20006,33351,28865,31572,20248,7768,18488,61054,209064,32153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"State Senate District 34 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):034",321252,157436,163816,161872,48047,1058,22331,0,77737,10207,...,19058,29218,24044,31574,23043,9317,33237,48533,204034,49978
"State Senate District 35 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):035",301542,145250,156292,175189,57545,918,21284,66,36125,10415,...,17887,27945,22304,24275,18990,8756,48501,53054,186220,42745
"State Senate District 36 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):036",325191,146459,178732,47342,198884,1297,10095,48,57713,9812,...,20781,34816,29242,34730,21409,8217,16314,58280,204993,39735
"State Senate District 37 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):037",314509,151213,163296,239728,20967,402,17666,35,28430,7281,...,15862,25487,21038,24365,22245,10722,61741,55424,190926,49611


In [14]:
ny_senate_demo_data_2012 = ny_senate_demo_data_2012.reset_index(drop=False)

ny_senate_demo_data_2012

,index,Total Pop,Male,Female,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,...,"$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $64,999","$65,000 to $74,999","$75,000 or more",5 to 17 years,18 to 64 years,65 years and over
0,"State Senate District 40 (2012), New York: Sum...",304418,150390,154028,253024,15252,622,10588,27,17849,...,14996,25047,21186,23325,20229,10449,62704,57746,188432,42036
1,"State Senate District 8 (2012), New York: Summ...",312213,150938,161275,227510,50463,934,9875,9,16133,...,17663,27642,24005,29723,23567,11550,48856,55467,195262,43341
2,"State Senate District 39 (2012), New York: Sum...",293597,145730,147867,229963,29964,689,7720,20,18671,...,18094,27908,22690,24429,21008,10466,35704,59077,180984,31151
3,"State Senate District 9 (2012), New York: Summ...",312806,151273,161533,232591,37889,644,18038,9,17437,...,16376,26816,21828,27145,25529,11300,57567,55381,194032,47158
4,"State Senate District 10 (2012), New York: Sum...",323789,151850,171939,48425,169970,1496,40305,141,50220,...,20006,33351,28865,31572,20248,7768,18488,61054,209064,32153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,"State Senate District 34 (2012), New York: Sum...",321252,157436,163816,161872,48047,1058,22331,0,77737,...,19058,29218,24044,31574,23043,9317,33237,48533,204034,49978
59,"State Senate District 35 (2012), New York: Sum...",301542,145250,156292,175189,57545,918,21284,66,36125,...,17887,27945,22304,24275,18990,8756,48501,53054,186220,42745
60,"State Senate District 36 (2012), New York: Sum...",325191,146459,178732,47342,198884,1297,10095,48,57713,...,20781,34816,29242,34730,21409,8217,16314,58280,204993,39735
61,"State Senate District 37 (2012), New York: Sum...",314509,151213,163296,239728,20967,402,17666,35,28430,...,15862,25487,21038,24365,22245,10722,61741,55424,190926,49611


In [15]:
# u for upper, l for lower
ny_senate_demo_data_2012['index'] = ny_senate_demo_data_2012['index'].apply(lambda x: 'u'+x.geo[1][1])
ny_senate_demo_data_2012 = ny_senate_demo_data_2012.rename(columns={'index': 'district'})
ny_senate_demo_data_2012

,district,Total Pop,Male,Female,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,...,"$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $64,999","$65,000 to $74,999","$75,000 or more",5 to 17 years,18 to 64 years,65 years and over
0,u040,304418,150390,154028,253024,15252,622,10588,27,17849,...,14996,25047,21186,23325,20229,10449,62704,57746,188432,42036
1,u008,312213,150938,161275,227510,50463,934,9875,9,16133,...,17663,27642,24005,29723,23567,11550,48856,55467,195262,43341
2,u039,293597,145730,147867,229963,29964,689,7720,20,18671,...,18094,27908,22690,24429,21008,10466,35704,59077,180984,31151
3,u009,312806,151273,161533,232591,37889,644,18038,9,17437,...,16376,26816,21828,27145,25529,11300,57567,55381,194032,47158
4,u010,323789,151850,171939,48425,169970,1496,40305,141,50220,...,20006,33351,28865,31572,20248,7768,18488,61054,209064,32153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,u034,321252,157436,163816,161872,48047,1058,22331,0,77737,...,19058,29218,24044,31574,23043,9317,33237,48533,204034,49978
59,u035,301542,145250,156292,175189,57545,918,21284,66,36125,...,17887,27945,22304,24275,18990,8756,48501,53054,186220,42745
60,u036,325191,146459,178732,47342,198884,1297,10095,48,57713,...,20781,34816,29242,34730,21409,8217,16314,58280,204993,39735
61,u037,314509,151213,163296,239728,20967,402,17666,35,28430,...,15862,25487,21038,24365,22245,10722,61741,55424,190926,49611


Get all the required census data

In [113]:
census_data = pd.DataFrame()
# acs data doesn't exist for census years ie 2020
for year in [2012, 2014, 2016, 2018]:
    demo_data_senate = censusdata.download('acs5', year, censusdata.censusgeo([('state', '36'), ('state%20legislative%20district%20(upper%20chamber)', '*')]), [each[:6] + '_' + each[6:] + 'E' for each in values_to_grab.keys()])
    demo_data_assembly = censusdata.download('acs5', year, censusdata.censusgeo([('state', '36'), ('state%20legislative%20district%20(lower%20chamber)', '*')]), [each[:6] + '_' + each[6:] + 'E' for each in values_to_grab.keys()])
    
    demo_data_senate = demo_data_senate.rename(columns=dict(zip([each[:6] + '_' + each[6:] + 'E' for each in values_to_grab.keys()], values_to_grab.values())))
    demo_data_assembly = demo_data_assembly.rename(columns=dict(zip([each[:6] + '_' + each[6:] + 'E' for each in values_to_grab.keys()], values_to_grab.values())))
    
    demo_data_assembly = demo_data_assembly.reset_index(drop=False)
    demo_data_senate = demo_data_senate.reset_index(drop=False)
    
    demo_data_assembly['index'] = demo_data_assembly['index'].apply(lambda x: 'l'+x.geo[1][1])
    demo_data_assembly = demo_data_assembly.rename(columns={'index': 'district'})

    demo_data_senate['index'] = demo_data_senate['index'].apply(lambda x: 'u'+x.geo[1][1])
    demo_data_senate = demo_data_senate.rename(columns={'index': 'district'})
    
    demo_data_assembly['year'] = year
    demo_data_senate['year'] = year
    
    census_data = pd.concat([census_data, demo_data_assembly, demo_data_senate])

census_data

,district,Total Pop,Male,Female,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,...,"$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $64,999","$65,000 to $74,999","$75,000 or more",5 to 17 years,18 to 64 years,65 years and over,year
0,l134,131534,63157,68377,118629,6482,300,2123,7,1886,...,15803,13897,16734,11775,4450,9640,21471,82456,20597,2012
1,l135,132025,63465,68560,123730,2534,94,3357,57,697,...,12630,11231,14188,11665,5490,17921,24352,78759,21820,2012
2,l064,124067,59521,64546,102746,3392,382,10947,76,4393,...,11927,9639,11586,9302,4343,17324,18874,79886,18525,2012
3,l065,128968,62745,66223,51937,6551,397,56030,62,10254,...,11237,8378,8673,8533,3967,21199,12008,92441,19082,2012
4,l058,129703,58039,71664,4714,118255,218,2188,8,2923,...,14523,12028,15282,9211,3499,7277,20507,84568,16437,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,u058,289885,143670,146215,257519,10623,646,11311,32,2207,...,33825,27206,29373,21529,8235,23473,41524,184105,49300,2018
59,u059,301552,153632,147920,276995,11158,1103,5691,51,2371,...,33261,29041,36581,27796,12027,30687,42909,192417,51959,2018
60,u060,293136,140555,152581,246681,19162,2330,7929,43,8790,...,31555,29493,34973,25680,10509,33103,43418,183418,50301,2018
61,u061,293562,142097,151465,235160,32303,1251,14299,57,3020,...,31030,25990,30079,23737,11788,38454,45967,179991,52619,2018


# 2020 census data is stored completely differently - come back to later

https://www.census.gov/programs-surveys/acs/data/experimental-data/1-year.html

In [16]:
# dataset is too large to be loaded to github but can be found at https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/3WZFK9

# harvard_data = pd.read_csv('196slers1967to2016_20180908.tab', delimiter='\t')

# ny_election_data = harvard_data[(harvard_data['sab'] == 'ny') & (harvard_data['year'] >= 2011)][['year', 'sab', 'ddez', 'sen', 'dname', 'dno', 'dtype', 'etype', 'cando', 'cand', 'party', 'prior1', 'prior2', 'exper', 'vote', 'outcome']]

In [17]:
harvard_data = pd.read_csv('harvard_data_2010+.csv', index_col=0)
harvard_data

/Users/stone/PycharmProjects/capstone/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2,13,14,18,23,28,32,48,56,67,71,72,75,76,77,78,79,80,81,82,83,84,85,86,87) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,caseid,uncert,year,month,day,sid,sab,sfips,sicpsr,cname,...,ltype2b,namebsource,v38,v39,v56,v57,v58,v18_20171211,v19_20171211,v19_20160217
3257,294964,NaN,2010,11.0,2.0,1,al,1,41,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,294964.0,"BURDINE, GREG","BURDINE, GREG"
3258,294965,NaN,2010,11.0,2.0,1,al,1,41,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2221.0,"HANSON, QUINTON","HANSON, QUINTON"
3259,294966,NaN,2010,11.0,2.0,1,al,1,41,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,294966.0,WRITEIN,WRITEIN
3260,294967,NaN,2010,11.0,2.0,1,al,1,41,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,270539.0,"CURTIS, MIKE","CURTIS, MIKE"
3261,294968,NaN,2010,11.0,2.0,1,al,1,41,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3249.0,"GREER, LYNN","GREER, LYNN"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378340,378228,NaN,2016,11.0,8.0,50,wy,56,68,natrona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,321320.0,"ANDERSON, JAMES LEE",NaN
378341,378230,NaN,2016,11.0,8.0,50,wy,56,68,natrona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,378230.0,SCATTERING,NaN
378342,378223,NaN,2016,11.0,8.0,50,wy,56,68,natrona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,378223.0,"FORD, ROBERT",NaN
378343,378225,NaN,2016,11.0,8.0,50,wy,56,68,natrona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256633.0,"SCOTT, CHARLES K.",NaN


In [18]:
ny_election_data = harvard_data[(harvard_data['sab'] == 'ny') & (harvard_data['year'] >= 2011)][['year', 'sab', 'ddez', 'sen', 'dname', 'dno', 'dtype', 'etype', 'cando', 'cand', 'party', 'prior1', 'prior2', 'exper', 'vote', 'outcome']]

In [19]:
ny_election_data['prior1'].value_counts()

Series([], Name: prior1, dtype: int64)

In [20]:
ny_election_data['prior2'].value_counts()

Series([], Name: prior2, dtype: int64)

In [21]:
ny_election_data['cand'].value_counts()

scattering                362
seward, james l.           69
lopez, peter d.            55
amedore, george a. jr.     51
little, elizabeth o.       42
                         ... 
meyer, mindy                1
langdon, andrew p.          1
koon, david r.              1
burke, richard e.           1
sweeney, michael j.         1
Name: cand, Length: 770, dtype: int64

In [22]:
ny_election_data['etype'].value_counts()

g                4248
dp                245
rp                181
conservativep     151
indepp            144
workp             136
greenp              5
Name: etype, dtype: int64

In [23]:
ny_election_data['dtype'].value_counts()

1.0    5110
Name: dtype, dtype: int64

In [24]:
ny_election_data['dname'].value_counts()

Series([], Name: dname, dtype: int64)

Prior columns are useless for this segment of data though incumbency (expr column) is going to be a huge indicator obviously. For this research I'm going to limit to general elections. Then I'm going to need to combine candidates due to NY's fusion voting system.

In [25]:
ny_election_data = ny_election_data[ny_election_data['etype'] == 'g'][['year', 'sab', 'ddez', 'sen', 'cand', 'party', 'exper', 'vote', 'outcome']]
ny_election_data

,year,sab,ddez,sen,cand,party,exper,vote,outcome
238120,2012,ny,1,0,"thiele, fred w. jr.",democrat,inc,29644.0,w
238121,2012,ny,1,0,"thiele, fred w. jr.",workingfamilies,inc,4142.0,w
238122,2012,ny,1,0,"thiele, fred w. jr.",independent,inc,4219.0,w
238123,2012,ny,2,0,"deegan, nicholas f.",democrat,none,18159.0,l
238124,2012,ny,2,0,"losquadro, daniel p.",independent,inc,1941.0,w
...,...,...,...,...,...,...,...,...,...
248553,2016,ny,63,1,"kennedy, timothy m.",democrat,inc,78674.0,w
248554,2016,ny,63,1,"kennedy, timothy m.",wep,inc,1191.0,w
248555,2016,ny,63,1,"kennedy, timothy m.",indep,inc,3751.0,w
248556,2016,ny,63,1,"kennedy, timothy m.",working,inc,6034.0,w


In [26]:
list_of_candidates = ny_election_data['cand'].unique()

list_of_candidates

array(['thiele, fred w. jr.', 'deegan, nicholas f.',
       'losquadro, daniel p.', 'hennessey, edward j.', 'murray, dean',
       'englebright, steven', 'mckee, deborah j.', 'salamone, victor e.',
       'graf, alfred c.', 'ramos, philip r.', 'troche, manuel',
       'bodkin, christopher d.', 'garbarino, andrew r.',
       'safer, jesse a.', 'fitzpatrick, michael j.', 'cherlin, jay s.',
       'saladino, joseph s.', 'dujmic, joseph s. jr.',
       'lupinacci, chad a.', 'sweeney, robert k.', 'cureton, rashad',
       'raia, andrew p.', 'lavine, charles d.', 'imbroto, louis',
       'peress, jeffery j.', 'brooks, john e.', 'mcdonough, david g.',
       'ferone, mario', 'montesano, michael a.', 'schimel, michelle e.',
       'stiek, richard e.', 'brady, kevin c.', 'mckevitt, thomas',
       'hooper, earlene hill', 'mccabe, elton e.', 'port, gary b.',
       'ra, edward p.', 'weisenberg, harvey', 'sussman, david j.',
       'friedman, jeffrey s.', 'curran, brian f.',
       'solages, mich

In [27]:
ny_election_data['year'].value_counts()

2016    2083
2014    1355
2012     810
Name: year, dtype: int64

no off year elections

In [28]:
combined_ballot_lines = []

for candidate in list_of_candidates:
    ballot_lines = ny_election_data[ny_election_data['cand'] == candidate]
    for year in ballot_lines['year'].unique():
        single_year = ballot_lines[ballot_lines['year'] == year].sort_values(by=['vote'], ascending=False)
        
        vote_total = single_year['vote'].sum()
        year_line = single_year.iloc[0]
#         print(year_line)
        year_line.at['vote'] = vote_total
        combined_ballot_lines.append(year_line)

df_combined_ballot_lines = pd.DataFrame(combined_ballot_lines)

df_combined_ballot_lines

/Users/stone/PycharmProjects/capstone/venv/lib/python3.8/site-packages/pandas/core/series.py:1169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_values(loc, value)


,year,sab,ddez,sen,cand,party,exper,vote,outcome
238120,2012,ny,1,0,"thiele, fred w. jr.",democrat,inc,38005.0,w
239258,2014,ny,1,0,"thiele, fred w. jr.",democrat,inc,19956.0,w
240067,2016,ny,1,0,"thiele, fred w. jr.",democrat,inc,35246.0,w
238123,2012,ny,2,0,"deegan, nicholas f.",democrat,none,18159.0,l
238126,2012,ny,2,0,"losquadro, daniel p.",modernrepublican,inc,34305.0,w
...,...,...,...,...,...,...,...,...,...
248502,2016,ny,60,1,"small, adam e.",democrat,none,51036.0,l
248503,2016,ny,60,1,"depasquale, james v.",green,none,2835.0,l
248509,2016,ny,60,1,"jacobs, christopher l.",republican,none,77327.0,w
248513,2016,ny,61,1,"loughran, thomas a.",democrat,none,53363.0,l


In [29]:
ny_election_data[ny_election_data['cand'] == 'thiele, fred w. jr.'].groupby(['year']).sum()

,sen,vote
year,,
2012,0,38005.0
2014,0,19956.0
2016,0,35246.0


In [30]:
df_combined_ballot_lines[(df_combined_ballot_lines['year'] == 2012) & (df_combined_ballot_lines['ddez'] == 3) & (df_combined_ballot_lines['sen'] == 1)]

,year,sab,ddez,sen,cand,party,exper,vote,outcome
246636,2012,ny,3,1,"genco, francis t.",democrat,none,41139.0,l
246639,2012,ny,3,1,"zeldin, lee m.",modernrepublican,inc,51920.0,w


In [31]:
ny_election_data[(ny_election_data['year'] == 2014) & (ny_election_data['ddez'] == 3) & (ny_election_data['sen'] == 1)]

,year,sab,ddez,sen,cand,party,exper,vote,outcome
247169,2014,ny,3,1,"esposito, adrienne",democrat,none,21213.0,l
247170,2014,ny,3,1,"croci, thomas d.",independent,none,1776.0,w
247171,2014,ny,3,1,"croci, thomas d.",conservative,none,5681.0,w
247172,2014,ny,3,1,"esposito, adrienne",green,none,952.0,l
247173,2014,ny,3,1,"esposito, adrienne",workingfamilies,none,2035.0,l
247174,2014,ny,3,1,"croci, thomas d.",modernrepublican,none,26404.0,w


In [32]:
df_combined_ballot_lines['district'] = df_combined_ballot_lines.apply(lambda row: ('u' if row['sen'] == 1 else 'l') + '{:0>3}'.format(row['ddez']), axis=1)
df_combined_ballot_lines

,year,sab,ddez,sen,cand,party,exper,vote,outcome,district
238120,2012,ny,1,0,"thiele, fred w. jr.",democrat,inc,38005.0,w,l001
239258,2014,ny,1,0,"thiele, fred w. jr.",democrat,inc,19956.0,w,l001
240067,2016,ny,1,0,"thiele, fred w. jr.",democrat,inc,35246.0,w,l001
238123,2012,ny,2,0,"deegan, nicholas f.",democrat,none,18159.0,l,l002
238126,2012,ny,2,0,"losquadro, daniel p.",modernrepublican,inc,34305.0,w,l002
...,...,...,...,...,...,...,...,...,...,...
248502,2016,ny,60,1,"small, adam e.",democrat,none,51036.0,l,u060
248503,2016,ny,60,1,"depasquale, james v.",green,none,2835.0,l,u060
248509,2016,ny,60,1,"jacobs, christopher l.",republican,none,77327.0,w,u060
248513,2016,ny,61,1,"loughran, thomas a.",democrat,none,53363.0,l,u061


In [33]:
df_combined_ballot_lines[df_combined_ballot_lines['sen'] == 1]['ddez'].unique()

array([ 8, 18, 26, 31, 28, 33, 49, 63,  9, 20, 21, 25, 19, 29, 56, 55,  1,
       27,  2,  3,  4,  5,  6,  7, 10, 11, 12, 13, 14, 15, 16, 17, 22, 23,
       24, 30, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
       48, 50, 51, 52, 53, 54, 57, 58, 59, 60, 61, 62])

In [34]:
df_combined_ballot_lines[(df_combined_ballot_lines['sen'] == 1) & (df_combined_ballot_lines['year'] == 2016) & (df_combined_ballot_lines['ddez'] == 63)]

,year,sab,ddez,sen,cand,party,exper,vote,outcome,district
248553,2016,ny,63,1,"kennedy, timothy m.",democrat,inc,89650.0,w,u063


In [35]:
df_combined_ballot_lines[(df_combined_ballot_lines['sen'] == 0) & (df_combined_ballot_lines['year'] == 2016) & (df_combined_ballot_lines['ddez'] == 63)]

,year,sab,ddez,sen,cand,party,exper,vote,outcome,district
240384,2016,ny,63,0,"cusick, michael j.",democrat,inc,34559.0,w,l063


In [36]:
df_combined_ballot_lines['ddez'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150])

In [37]:
((df_combined_ballot_lines['ddez'] == 150) & (df_combined_ballot_lines['sen'] == 1)).any()

False

In [38]:
df_combined_ballot_lines[(df_combined_ballot_lines['year'] == 2014) & (df_combined_ballot_lines['ddez'] == 3) & (df_combined_ballot_lines['sen'] == 1)]

,year,sab,ddez,sen,cand,party,exper,vote,outcome,district
247169,2014,ny,3,1,"esposito, adrienne",democrat,none,24200.0,l,u003
247174,2014,ny,3,1,"croci, thomas d.",modernrepublican,none,33861.0,w,u003


Need to combine rows so each row represents one general election. Vote feature should be net democratic votes


In [39]:
df_combined_ballot_lines = df_combined_ballot_lines[['year', 'cand', 'party', 'exper', 'vote', 'district']]

In [40]:
df_combined_ballot_lines[(df_combined_ballot_lines['year'] == 2012) & (df_combined_ballot_lines['district'] == 'l002')& (df_combined_ballot_lines['party'].isin(['democrat', 'modernrepublican']))]

,year,cand,party,exper,vote,district
238123,2012,"deegan, nicholas f.",democrat,none,18159.0,l002
238126,2012,"losquadro, daniel p.",modernrepublican,inc,34305.0,l002


In [41]:
def process_ny_boe_data(document_name):
    output = pd.DataFrame()

    for district in pd.read_excel('ny_data/' + document_name, sheet_name=None).keys():
        sheet = pd.read_excel('ny_data/' + document_name, sheet_name=district, header=1)
        if 'Candidate Name (Party)' in sheet.columns:
#             print(sheet['Candidate Name (Party)'])
            sheet = sheet.dropna()
            sheet['party'] = sheet['Candidate Name (Party)'].apply(lambda x: 'democratic' if x[-4:-1] == 'DEM' else ('modernrepublican' if x[-4:-1] == 'REP' else None))
            sheet = sheet[['Candidate Name (Party)', 'Total Votes by Candidate', 'party']]
            dnumber, dtype = district.split()
            dnumber = re.findall(r'\d+', dnumber)[0]
            if dtype == 'AD':
                district = 'l' + '{:0>3}'.format(dnumber)
            elif dtype == 'SD':
                district = 'u' + '{:0>3}'.format(dnumber)
            sheet['district'] = pd.Series([district, district])
            year = int(document_name[:4])
            sheet['year'] = pd.Series([year, year])
            output = pd.concat([output, sheet])
        else:
            pass
        
        
    output = output.rename(columns={'Candidate Name (Party)': 'cand', 'Total Votes by Candidate': "vote"})
    return (output)

In [42]:
# convert multisheet xlsx to easy to ammend csv
for file in os.listdir('ny_data/'):
    print(file)
    if os.path.isfile('ny_data/' + file):
        process_ny_boe_data(file).to_csv('ny_data/processed/' + file[:-5] + 'processed.csv')


.DS_Store


ValueError: Excel file format cannot be determined, you must specify an engine manually.

At this point I went in by hand and marked whether a candidate was an incumbent or not. Now we need to import that data back in, combine it with the 2012-2016 data, and then alter the info so that each election takes up one row.

In [43]:
ny_boe_data = pd.DataFrame()

for file in os.listdir('ny_data/processed/'):
#     print(file[-3:])
    if file[-4:] == '.csv':
        x = pd.read_csv('ny_data/processed/' + file, index_col=0)
        ny_boe_data = pd.concat([ny_boe_data, x])
        
# fill in NaNs in exper column with "none" to match harvard data
ny_boe_data['exper'] = ny_boe_data['exper'].fillna('none')
        

ny_boe_data

,cand,vote,party,district,year,exper
0,Laura A. Ahearn (DEM),82900,democratic,u001,2020.0,none
1,Anthony H. Palumbo (REP),87563,modernrepublican,u001,2020.0,none
0,Michael G. Siderakis (DEM),70833,democratic,u002,2020.0,none
1,Mario R. Mattera (REP),93224,modernrepublican,u002,2020.0,none
0,Monica R. Martinez (DEM),63286,democratic,u003,2020.0,inc
...,...,...,...,...,...,...
1,Joseph M. Giglio (REP),39964,modernrepublican,l148,2020.0,inc
0,Jonathan D. Rivera (DEM),38611,democratic,l149,2020.0,none
1,Joseph Totaro (REP),19922,modernrepublican,l149,2020.0,none
0,Christina Cardinale (DEM),17845,democratic,l150,2020.0,none


In [56]:
df_combined_ballot_lines[(df_combined_ballot_lines['year'] == 2016) & (df_combined_ballot_lines['district'] == 'l003')]

,year,cand,party,exper,vote,district
240083,2016,"murray, dean",republican,inc,29087.0,l003
240079,2016,"schoen, gregory d.",democrat,none,17404.0,l003


In [67]:
df_combined_ballot_lines = pd.concat([df_combined_ballot_lines, ny_boe_data])

df_combined_ballot_lines

,year,cand,party,exper,vote,district
238120,2012.0,"thiele, fred w. jr.",democrat,inc,38005.0,l001
239258,2014.0,"thiele, fred w. jr.",democrat,inc,19956.0,l001
240067,2016.0,"thiele, fred w. jr.",democrat,inc,35246.0,l001
238123,2012.0,"deegan, nicholas f.",democrat,none,18159.0,l002
238126,2012.0,"losquadro, daniel p.",modernrepublican,inc,34305.0,l002
...,...,...,...,...,...,...
1,2020.0,Joseph M. Giglio (REP),modernrepublican,inc,39964.0,l148
0,2020.0,Jonathan D. Rivera (DEM),democratic,none,38611.0,l149
1,2020.0,Joseph Totaro (REP),modernrepublican,none,19922.0,l149
0,2020.0,Christina Cardinale (DEM),democratic,none,17845.0,l150


combine republican and modernrepublican, democrat and democratic

In [75]:
df_combined_ballot_lines = df_combined_ballot_lines.replace('republican', 'modernrepublican')
df_combined_ballot_lines = df_combined_ballot_lines.replace('democrat', 'democratic')

In [74]:
df_combined_ballot_lines['party'].unique()

array(['democrat', 'modernrepublican', 'green', 'conservative',
       'independent', 'unspecifiednonmajor', 'libertarian',
       'workingfamilies', 'scattering', 'nip', 'wep', 'working', 'indep',
       'rpp', 'reform', 'democratic'], dtype=object)

## features should be:
- derive from election data
 - percentage of votes for democrat (excluding third party, etc) - target
 - was democrat incumbent
 - was republican incumbent
 - 

- copied from election data
 - year
 - district
 
- census data

- national generic ballot (can get from RCP)

In [ ]:
df_combined_ballot_lines[df_combined_ballot_lines['party'] == 'republican']

In [ ]:
df_combined_ballot_lines[(df_combined_ballot_lines['year'] == 2016)&(df_combined_ballot_lines['district'] == 'l048')]

In [102]:
# third party ballot lines have already been eliminated. this is to convert 2 line elections into 1 line

years = df_combined_ballot_lines['year'].unique()
districts = df_combined_ballot_lines['district'].unique()
final_election_data = pd.DataFrame()

for year in years:
    for district in districts:
        election_df = df_combined_ballot_lines[(df_combined_ballot_lines['year'] == year) &
                                                (df_combined_ballot_lines['district'] == district)]
        
        if election_df[election_df['party'] == 'democratic'].empty:
            dem_incumbent = False
            dem_net_vote = numpy.nan
            rep_incumbent = election_df[election_df['party'] == 'modernrepublican']['exper'].iloc[0]
            if rep_incumbent == 'inc':
                rep_incumbent = True
            else:
                rep_incumbent = False
        elif election_df[election_df['party'] == 'modernrepublican'].empty:
            rep_incumbent = False
            dem_net_vote = numpy.nan
            dem_incumbent = election_df[election_df['party'] == 'democratic']['exper'].iloc[0]
            if dem_incumbent == 'inc':
                dem_incumbent = True
            else:
                dem_incumbent = False
        else:
        
            dem_net_vote = election_df[(election_df['party'] == 'democratic')]['vote'].iloc[0] - election_df[(election_df['party'] == 'modernrepublican')]['vote'].iloc[0]


            dem_incumbent = election_df[election_df['party'] == 'democratic']['exper'].iloc[0]
            if dem_incumbent == 'inc':
                dem_incumbent = True
            else:
                dem_incumbent = False
            rep_incumbent = election_df[election_df['party'] == 'modernrepublican']['exper'].iloc[0]
            if rep_incumbent == 'inc':
                rep_incumbent = True
            else:
                rep_incumbent = False
    
        final_election_data = final_election_data.append({'year': year,
                                'district': district,
                                'dem_net_vote': dem_net_vote,
                                'dem_incumbent': dem_incumbent,
                                'rep_incumbent': rep_incumbent
                               }, ignore_index=True)
            
final_election_data

,year,district,dem_net_vote,dem_incumbent,rep_incumbent
0,2012.0,l001,NaN,1.0,0.0
1,2012.0,l002,-16146.0,0.0,1.0
2,2012.0,l003,239.0,0.0,1.0
3,2012.0,l004,11577.0,1.0,0.0
4,2012.0,l005,-7310.0,0.0,1.0
...,...,...,...,...,...
1060,2018.0,u058,-18673.0,0.0,1.0
1061,2018.0,u059,NaN,0.0,1.0
1062,2018.0,u060,-12744.0,0.0,1.0
1063,2018.0,u061,-9309.0,0.0,1.0


# Removing 2020 results due to lack of census data

In [109]:
final_election_data = final_election_data[final_election_data['year'].isin([2012, 2014, 2016, 2018])]
final_election_data

,year,district,dem_net_vote,dem_incumbent,rep_incumbent
0,2012.0,l001,NaN,1.0,0.0
1,2012.0,l002,-16146.0,0.0,1.0
2,2012.0,l003,239.0,0.0,1.0
3,2012.0,l004,11577.0,1.0,0.0
4,2012.0,l005,-7310.0,0.0,1.0
...,...,...,...,...,...
1060,2018.0,u058,-18673.0,0.0,1.0
1061,2018.0,u059,NaN,0.0,1.0
1062,2018.0,u060,-12744.0,0.0,1.0
1063,2018.0,u061,-9309.0,0.0,1.0


In [126]:
sample = pd.merge(left = final_election_data,
         right = census_data,
        how = 'left',
        on = ['district', 'year'])
sample

,year,district,dem_net_vote,dem_incumbent,rep_incumbent,Total Pop,Male,Female,White alone,Black or African American alone,...,"$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $64,999","$65,000 to $74,999","$75,000 or more",5 to 17 years,18 to 64 years,65 years and over
0,2012.0,l001,NaN,1.0,0.0,128864,65218,63646,112936,6578,...,7510,13272,10642,12701,9182,5288,20309,20610,80163,21410
1,2012.0,l002,-16146.0,0.0,1.0,125986,61397,64589,115303,4177,...,7157,11315,10044,11740,10047,5204,22357,20553,74496,23499
2,2012.0,l003,239.0,0.0,1.0,129515,63192,66323,108201,11606,...,6860,12189,10018,13421,11070,4762,14870,23420,83478,13824
3,2012.0,l004,11577.0,1.0,0.0,128333,63694,64639,102795,7567,...,6130,11710,8766,10354,8516,4879,21665,22277,82614,16207
4,2012.0,l005,-7310.0,0.0,1.0,130292,64977,65315,116397,2877,...,7776,11706,9629,12619,10676,5650,18883,24649,83642,14932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,2018.0,u058,-18673.0,0.0,1.0,289885,143670,146215,257519,10623,...,21761,33825,27206,29373,21529,8235,23473,41524,184105,49300
848,2018.0,u059,NaN,0.0,1.0,301552,153632,147920,276995,11158,...,19828,33261,29041,36581,27796,12027,30687,42909,192417,51959
849,2018.0,u060,-12744.0,0.0,1.0,293136,140555,152581,246681,19162,...,20534,31555,29493,34973,25680,10509,33103,43418,183418,50301
850,2018.0,u061,-9309.0,0.0,1.0,293562,142097,151465,235160,32303,...,19624,31030,25990,30079,23737,11788,38454,45967,179991,52619


In [127]:
sample = sample.dropna(subset=['dem_net_vote'])

In [128]:
sample

,year,district,dem_net_vote,dem_incumbent,rep_incumbent,Total Pop,Male,Female,White alone,Black or African American alone,...,"$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $64,999","$65,000 to $74,999","$75,000 or more",5 to 17 years,18 to 64 years,65 years and over
1,2012.0,l002,-16146.0,0.0,1.0,125986,61397,64589,115303,4177,...,7157,11315,10044,11740,10047,5204,22357,20553,74496,23499
2,2012.0,l003,239.0,0.0,1.0,129515,63192,66323,108201,11606,...,6860,12189,10018,13421,11070,4762,14870,23420,83478,13824
3,2012.0,l004,11577.0,1.0,0.0,128333,63694,64639,102795,7567,...,6130,11710,8766,10354,8516,4879,21665,22277,82614,16207
4,2012.0,l005,-7310.0,0.0,1.0,130292,64977,65315,116397,2877,...,7776,11706,9629,12619,10676,5650,18883,24649,83642,14932
5,2012.0,l006,17148.0,1.0,0.0,130598,65399,65199,66302,24827,...,9088,17967,12379,13638,8692,2722,7314,24889,84901,11359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,2018.0,u053,13428.0,0.0,0.0,285981,137846,148135,221349,40799,...,20598,32026,26882,30787,21617,9038,23234,42179,183689,43614
845,2018.0,u054,-25954.0,0.0,1.0,293440,144683,148757,269550,9029,...,20271,35972,30821,34644,24533,10559,28317,45737,178626,53362
847,2018.0,u058,-18673.0,0.0,1.0,289885,143670,146215,257519,10623,...,21761,33825,27206,29373,21529,8235,23473,41524,184105,49300
849,2018.0,u060,-12744.0,0.0,1.0,293136,140555,152581,246681,19162,...,20534,31555,29493,34973,25680,10509,33103,43418,183418,50301


let's add in generic ballot numbers

In [121]:
generic_ballot = {2012: -0.2,
                  2014: -2.4,
                  2016: 0.6,
                  2018: 7.3,
                  2020: 6.8}

In [134]:
sample['generic_ballot'] = sample.apply(lambda row: generic_ballot[row['year']], axis=1)

/var/folders/gz/fthcl8n50xb9x3ts2d8zdp3w0000gn/T/ipykernel_11046/3247741581.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['generic_ballot'] = sample.apply(lambda row: generic_ballot[row['year']], axis=1)


let's count the number of incumbent losses

In [137]:
flips = sample[((sample['dem_net_vote'] < 0) & (sample['dem_incumbent'] == True)) | ((sample['dem_net_vote'] > 0) & (sample['rep_incumbent'] == True))]

flips

,year,district,dem_net_vote,dem_incumbent,rep_incumbent,Total Pop,Male,Female,White alone,Black or African American alone,...,"$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $64,999","$65,000 to $74,999","$75,000 or more",5 to 17 years,18 to 64 years,65 years and over,generic_ballot
2,2012.0,l003,239.0,0.0,1.0,129515,63192,66323,108201,11606,...,12189,10018,13421,11070,4762,14870,23420,83478,13824,-0.2
98,2012.0,l093,4785.0,0.0,1.0,133035,64177,68858,105177,6476,...,9255,7789,8596,7623,4026,31057,25829,81964,17991,-0.2
133,2012.0,l127,6279.0,0.0,1.0,134037,64847,69190,124767,3209,...,13657,12744,16645,12536,5523,14665,24102,83739,18442,-0.2
194,2012.0,u041,2096.0,0.0,1.0,309254,152924,156330,247872,28330,...,29436,26135,30876,24217,11633,40488,52735,197845,42989,-0.2
215,2014.0,l003,-524.0,1.0,0.0,128172,62893,65279,105805,12074,...,12474,10012,12502,11016,4953,15020,22207,82489,15034,-2.4
378,2014.0,u055,-16250.0,1.0,0.0,294963,143190,151773,240410,32753,...,31561,27710,31939,22856,10254,35542,49132,184212,45469,-2.4
407,2014.0,u041,-3642.0,1.0,0.0,309349,153235,156114,247148,29132,...,30970,26289,29808,23634,10874,42760,50334,198098,45874,-2.4
440,2016.0,u008,314.0,0.0,1.0,316165,154613,161552,220956,51036,...,29108,25828,27701,24476,10801,53452,52149,199633,45425,0.6
578,2016.0,l145,-4772.0,1.0,0.0,128761,62386,66375,108263,12706,...,16464,12354,13375,10099,4147,10885,20402,79255,22457,0.6
647,2018.0,l009,-6046.0,1.0,0.0,127020,61667,65353,113307,4103,...,10576,9431,11281,10934,4814,27245,19999,79124,21949,7.3


In [138]:
flips['district'].value_counts()

l003    2
u041    2
l121    1
u022    1
u007    1
u006    1
u005    1
l146    1
l142    1
l116    1
l093    1
l021    1
l009    1
l145    1
u008    1
u055    1
l127    1
u040    1
Name: district, dtype: int64